### Chargement des données

In [11]:
import pandas as pd
from io import StringIO
import os 

# ATTENTION : Remplacer toutes les virgules par des points dans les fichiers csv
folder_path = '/workspace/ocealia/data'

df_dict = {}

# Read all CSV files in the folder and assign them to variables based on filenames
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        var_name = os.path.splitext(filename)[0]
        df= pd.read_csv(os.path.join(folder_path, filename),sep=';')
        globals()[var_name+'_df'] = df
        df_dict[var_name] = df

for name, df in df_dict.items():
    print(name)
    print(*df.columns.tolist(), sep="\n")
    print("\n")



pd.set_option('display.max_columns', None)



cellule
CODE_SITE
CODE_CELLULE
TYPE_CELLULE
VENTILATION
CAPACITE


cout_transport
CODE_SITE_ORIGINE
CODE_SITE_DESTINATION
COUT


out_capacite_stockage
CODE_PRODUIT_STOCKAGE
CODE_SITE
CAPACITE_DE_STOCKAGE
PERIODE


out_collecte_prev_eq_ble
CODE_PRODUIT_STOCKAGE
CODE_SITE
MASSE_VOL
TYPE
PERIODE
QTE_PREV
QTE_PREV_EQ_BLE


out_cout_transfert
CODE_PRODUIT
CODE_SITE_ORIGINE
CODE_SITE_DESTINATION
QUANTITE_TRANSFEREE
COUT_UNITAIRE
COUT_TRANSFERT


out_quantite_a_stocker
CODE_PRODUIT_STOCKAGE
CODE_SITE
QUANTITE_A_STOCKER


out_reste_a_stocker
CODE_PRODUIT_STOCKAGE
CODE_SITE
QUANTITE_A_STOCKER
CAPACITE_DE_STOCKAGE
PERIODE
RESTE_A_STOCKER


plan_affectation
CODE_DIVISION
CODE_CELLULE
CODE_PRODUIT_STOCKAGE_ETE
CODE_PRODUIT_STOCKAGE_AUTOMNE


plan_transfert
CODE_PRODUIT
CODE_SITE_ORIGINE
CODE_SITE_DESTINATION
QUANTITE_TRANSFEREE


prevision_collecte
CODE_PRODUIT
CODE_ADHERENT
CODE_SITE
SURFACE_PREVISIONNELLE


produit
CODE_PRODUIT
RENDEMENT
CODE_PRODUIT_STOCKAGE
PRODUIT
PERIODE
TYPE
MASSE_VOL


sit

### 1.	COLLECTE PREV : Prévision Ha par adhérent à Prévision TO par site

In [14]:
collecte_prevision_df = prevision_collecte_df.merge(produit_df, on='CODE_PRODUIT', how='left')

print(collecte_prevision_df.columns.tolist())

collecte_prevision_df['SURFACE_PREVISIONNELLE'] = pd.to_numeric(collecte_prevision_df['SURFACE_PREVISIONNELLE'], errors='coerce')
collecte_prevision_df['RENDEMENT'] = pd.to_numeric(collecte_prevision_df['RENDEMENT'], errors='coerce')

collecte_prevision_df['QTE_PREV'] = (collecte_prevision_df['SURFACE_PREVISIONNELLE'] * collecte_prevision_df['RENDEMENT'])

collecte_prevision_df = collecte_prevision_df.dropna()

['CODE_PRODUIT', 'CODE_ADHERENT', 'CODE_SITE', 'SURFACE_PREVISIONNELLE', 'RENDEMENT', 'CODE_PRODUIT_STOCKAGE', 'PRODUIT', 'PERIODE', 'TYPE', 'MASSE_VOL']


#### Verifications

In [15]:
collecte_prevision_df


,CODE_PRODUIT,CODE_ADHERENT,CODE_SITE,SURFACE_PREVISIONNELLE,RENDEMENT,CODE_PRODUIT_STOCKAGE,PRODUIT,PERIODE,TYPE,MASSE_VOL,QTE_PREV
0,112,301236,D520,7.000,1.0,CO1302,SARRASIN TRACE,AUT,CONV,1.15,7.000
1,112,392252,D620,3.990,1.0,CO1302,SARRASIN TRACE,AUT,CONV,1.15,3.990
2,112,387094,D520,30.040,1.0,CO1302,SARRASIN TRACE,AUT,CONV,1.15,30.040
4,112,387876,D053,2.000,1.0,CO1302,SARRASIN TRACE,AUT,CONV,1.15,2.000
5,112,390387,D678,6.860,1.0,CO1302,SARRASIN TRACE,AUT,CONV,1.15,6.860
...,...,...,...,...,...,...,...,...,...,...,...
12673,PC,394327,D669,2.538,2.0,CO1600,POIS CHICHE,SUM,CONV,1.00,5.076
12674,PC,394932,D631,7.030,2.0,CO1600,POIS CHICHE,SUM,CONV,1.00,14.060
12675,PC,2037508,D603,1.640,2.0,CO1600,POIS CHICHE,SUM,CONV,1.00,3.280
12676,PC,2037171,D503,1.500,2.0,CO1600,POIS CHICHE,SUM,CONV,1.00,3.000


In [5]:
filtered_df = collecte_prevision_df[(collecte_prevision_df['CODE_PRODUIT_STOCKAGE'] == 'CO0100') & (collecte_prevision_df['CODE_SITE'] == 'D001')]
filtered_df

,CODE_PRODUIT,CODE_ADHERENT,CODE_SITE,SURFACE_PREVISIONNELLE,RENDEMENT,CODE_PRODUIT_STOCKAGE,PRODUIT,PERIODE,TYPE,MASSE_VOL,QTE_PREV
653,CO0100,12780,D001,21.260,5.9,CO0100,BLE TENDRE,SUM,CONV,1.0,125.4340
1850,CO0100,21055,D001,4.665,5.9,CO0100,BLE TENDRE,SUM,CONV,1.0,27.5235
1882,CO0100,2037083,D001,22.000,5.9,CO0100,BLE TENDRE,SUM,CONV,1.0,129.8000


#### Agrégation

In [16]:
collecte_prev_agg_df = collecte_prevision_df.groupby(['CODE_PRODUIT_STOCKAGE', 'CODE_SITE', 'MASSE_VOL', 'TYPE', 'PERIODE'])['QTE_PREV'].sum().reset_index()


In [17]:
collecte_prev_agg_df

,CODE_PRODUIT_STOCKAGE,CODE_SITE,MASSE_VOL,TYPE,PERIODE,QTE_PREV
0,CO0100,D001,1.00,CONV,SUM,282.7575
1,CO0100,D002,1.00,CONV,SUM,138.1426
2,CO0100,D003,1.00,CONV,SUM,1147.3140
3,CO0100,D006,1.00,CONV,SUM,554.1457
4,CO0100,D007,1.00,CONV,SUM,1306.4075
...,...,...,...,...,...,...
2252,ME050,D602,1.00,AB,SUM,64.5000
2253,ME080,D201,1.15,AB,SUM,4.3800
2254,ME080,D439,1.15,AB,SUM,6.0000
2255,ME080,D456,1.15,AB,SUM,7.6200


### 2.	Calcul du coût de transfert (Equivalent Blé)

In [20]:
collecte_prev_agg_df['QTE_PREV'] = pd.to_numeric(collecte_prev_agg_df['QTE_PREV'])
collecte_prev_agg_df['MASSE_VOL'] = pd.to_numeric(collecte_prev_agg_df['MASSE_VOL'])
collecte_prev_agg_df['QTE_PREV_EQ_BLE'] = (collecte_prev_agg_df['QTE_PREV'] / collecte_prev_agg_df['MASSE_VOL']).round(0)
collecte_prev_agg_df

,CODE_PRODUIT_STOCKAGE,CODE_SITE,MASSE_VOL,TYPE,PERIODE,QTE_PREV,QTE_PREV_EQ_BLE
0,CO0100,D001,1.00,CONV,SUM,282.7575,283.0
1,CO0100,D002,1.00,CONV,SUM,138.1426,138.0
2,CO0100,D003,1.00,CONV,SUM,1147.3140,1147.0
3,CO0100,D006,1.00,CONV,SUM,554.1457,554.0
4,CO0100,D007,1.00,CONV,SUM,1306.4075,1306.0
...,...,...,...,...,...,...,...
2252,ME050,D602,1.00,AB,SUM,64.5000,64.0
2253,ME080,D201,1.15,AB,SUM,4.3800,4.0
2254,ME080,D439,1.15,AB,SUM,6.0000,5.0
2255,ME080,D456,1.15,AB,SUM,7.6200,7.0


In [8]:
collecte_prev_agg_df.to_csv('/workspace/ocealia/data/out_collecte_prev_eq_ble.csv', index=False, sep=';')

### 3.	Calcul du coût de transfert

In [21]:
cout_transport_df

,CODE_SITE_ORIGINE,CODE_SITE_DESTINATION,COUT
0,AQ07,D001,0.0
1,AQ07,D002,0.0
2,AQ07,D003,0.0
3,AQ07,D004,0.0
4,AQ07,D005,0.0
...,...,...,...
83516,DSAIP,DAMR,0.0
83517,DSAIP,DSAIP,0.0
83518,DSAIP,D619,0.0
83519,DSAIP,AQ07,0.0


In [22]:
plan_transfert_df['QUANTITE_TRANSFEREE'] = pd.to_numeric(plan_transfert_df['QUANTITE_TRANSFEREE'], errors='coerce')
cout_transport_df['COUT_UNITAIRE'] = pd.to_numeric(cout_transport_df['COUT'], errors='coerce')
cout_transport_df.drop(columns=['COUT'], inplace=True)
cout_transport_df


,CODE_SITE_ORIGINE,CODE_SITE_DESTINATION,COUT_UNITAIRE
0,AQ07,D001,0.0
1,AQ07,D002,0.0
2,AQ07,D003,0.0
3,AQ07,D004,0.0
4,AQ07,D005,0.0
...,...,...,...
83516,DSAIP,DAMR,0.0
83517,DSAIP,DSAIP,0.0
83518,DSAIP,D619,0.0
83519,DSAIP,AQ07,0.0


In [23]:
cout_transfert_df = pd.merge(plan_transfert_df, cout_transport_df, on=['CODE_SITE_ORIGINE', 'CODE_SITE_DESTINATION'], how='left')
cout_transfert_df['COUT_TRANSFERT'] = (cout_transfert_df['QUANTITE_TRANSFEREE'] * cout_transfert_df['COUT_UNITAIRE']).round(0)
cout_transfert_df

,CODE_PRODUIT,CODE_SITE_ORIGINE,CODE_SITE_DESTINATION,QUANTITE_TRANSFEREE,COUT_UNITAIRE,COUT_TRANSFERT
0,CO0501,D700,D707,180,7.06,1271.0
1,CO0501,D020,D631,42,3.43,144.0
2,CO0501,D404,D631,90,3.06,275.0
3,CO0501,D701,D707,46,5.20,239.0
4,CO0501,D702,D707,9,5.20,47.0
...,...,...,...,...,...,...
641,CO0700,D108,D201,135,3.06,413.0
642,CO0700,D511,D508,260,6.70,1742.0
643,CO0700,D681,D061,675,3.06,2066.0
644,CO0700,D056,D022,433,3.06,1325.0


In [39]:
cout_transfert_df.to_csv('/workspace/ocealia/data/out_cout_transfert.csv', index=False, sep=';')

##### Vérifications

In [ ]:
filtered_df = collecte_prevision_df[(collecte_prevision_df['CODE_PRODUIT_STOCKAGE'] == 'CO0100') & (collecte_prevision_df['CODE_SITE'] == 'D001')]
filtered_df

In [42]:
filtered_df = cout_transfert_df[(cout_transfert_df['CODE_SITE_ORIGINE'] == 'D700') & (cout_transfert_df['CODE_SITE_DESTINATION'] == 'D707')]
filtered_df

,CODE_SITE_ORIGINE,CODE_SITE_DESTINATION,COUT,CODE_PRODUIT,QUANTITE_TRANSFEREE,COUT_TRANSFERT
70874,D700,D707,7.06,CO0501,180.0,1270.80
70875,D700,D707,7.06,CO0504,47.0,331.82
70876,D700,D707,7.06,CO0502,14.0,98.84
70877,D700,D707,7.06,CO0700,187.0,1320.22


In [43]:
filtered_df = cout_transfert_df[(cout_transfert_df['CODE_SITE_ORIGINE'] == 'AQ07') & (cout_transfert_df['CODE_SITE_DESTINATION'] == 'D707')]
filtered_df

,CODE_SITE_ORIGINE,CODE_SITE_DESTINATION,COUT,CODE_PRODUIT,QUANTITE_TRANSFEREE,COUT_TRANSFERT
0,AQ07,D001,0.0,NaN,NaN,NaN


In [41]:
filtered_df = cout_transport_df[(cout_transport_df['CODE_SITE_ORIGINE'] == 'D700') & (cout_transport_df['CODE_SITE_DESTINATION'] == 'D707')]
filtered_df

,CODE_SITE_ORIGINE,CODE_SITE_DESTINATION,COUT
70824,D700,D707,7.06


In [ ]:
filtered_df = cout_transport_df[(cout_transport_df['CODE_SITE_ORIGINE'] == 'AQ07') & (cout_transport_df['CODE_SITE_DESTINATION'] == 'D001')]
filtered_df

In [36]:
filtered_df = cout_transport_df[(cout_transport_df['CODE_SITE_ORIGINE'] == 'D700') ]
filtered_df

,CODE_SITE_ORIGINE,CODE_SITE_DESTINATION,COUT
70805,D700,D228,0.0
70806,D700,D518,0.0
70807,D700,D700,0.0
70808,D700,D619,0.0
70809,D700,AQ07,0.0
...,...,...,...
71089,D700,D418,NaN
71090,D700,D460,NaN
71091,D700,D462,NaN
71092,D700,D476,NaN


In [ ]:
filtered_df = cout_transport_df[(cout_transport_df['CODE_SITE_ORIGINE'] == 'D700') & (cout_transport_df['CODE_SITE_DESTINATION'] == 'D707')]
filtered_df

In [35]:
cout_transport_df

,CODE_SITE_ORIGINE,CODE_SITE_DESTINATION,COUT
0,AQ07,D001,0.0
1,AQ07,D002,0.0
2,AQ07,D003,0.0
3,AQ07,D004,0.0
4,AQ07,D005,0.0
...,...,...,...
83516,DSAIP,DAMR,0.0
83517,DSAIP,DSAIP,0.0
83518,DSAIP,D619,0.0
83519,DSAIP,AQ07,0.0


In [49]:
filtered_df = plan_transfert_df[(plan_transfert_df['CODE_SITE_ORIGINE'] == 'AQ07') & (plan_transfert_df['CODE_SITE_DESTINATION'] == 'D001')]
filtered_df

,CODE_PRODUIT,CODE_SITE_ORIGINE,CODE_SITE_DESTINATION,QUANTITE_TRANSFEREE


### 4.	Construction du plan / Calcul capacité de stockage

In [27]:
cellule_df['CAPACITE'] = pd.to_numeric(cellule_df['CAPACITE'], errors='coerce')

# Left join Affectation and Cellule on CODE_SITE and CODE_CELLULE
#affectation_cellule_df = pd.merge(plan_affectation_df, cellule_df, on=['CODE_CELLULE'], how='left')
affectation_cellule_df = pd.merge(plan_affectation_df, cellule_df, left_on=['CODE_CELLULE', 'CODE_DIVISION'], right_on=['CODE_CELLULE', 'CODE_SITE'], how='left')






In [28]:
# Aggregation for ETE
stockage_ete_df = affectation_cellule_df.groupby(['CODE_PRODUIT_STOCKAGE_ETE', 'CODE_SITE'])['CAPACITE'].sum().reset_index()
stockage_ete_df.columns = ['CODE_PRODUIT_STOCKAGE', 'CODE_SITE', 'CAPACITE_DE_STOCKAGE']
stockage_ete_df['PERIODE'] = 'ETE'

# Aggregation for AUTOMNE
stockage_automne_df = affectation_cellule_df.groupby(['CODE_PRODUIT_STOCKAGE_AUTOMNE', 'CODE_SITE'])['CAPACITE'].sum().reset_index()
stockage_automne_df.columns = ['CODE_PRODUIT_STOCKAGE', 'CODE_SITE', 'CAPACITE_DE_STOCKAGE']
stockage_automne_df['PERIODE'] = 'AUT'

capacite_stockage_df = pd.concat([stockage_ete_df, stockage_automne_df], ignore_index=True)

In [29]:
capacite_stockage_df

,CODE_PRODUIT_STOCKAGE,CODE_SITE,CAPACITE_DE_STOCKAGE,PERIODE
0,CO0100,D001,1000.0,ETE
1,CO0100,D008,3668.0,ETE
2,CO0100,D010,1800.0,ETE
3,CO0100,D011,1500.0,ETE
4,CO0100,D012,8000.0,ETE
...,...,...,...,...
231,CO0802,D669,2900.0,AUT
232,CO0802,D674,1300.0,AUT
233,CO0802,D702,1250.0,AUT
234,CO1301,D674,2600.0,AUT


In [30]:
capacite_stockage_df.to_csv('/workspace/ocealia/data/out_capacite_stockage.csv', index=False, sep=';')

### 5.	Construction du plan / Quantité à stocker

In [26]:
cout_transfert_df.columns.to_list()

['CODE_PRODUIT',
 'CODE_SITE_ORIGINE',
 'CODE_SITE_DESTINATION',
 'QUANTITE_TRANSFEREE',
 'COUT',
 'COUT_TRANSFERT']

In [58]:
collecte_prev_agg_df.columns.to_list()

['CODE_PRODUIT_STOCKAGE',
 'CODE_SITE',
 'MASSE_VOL',
 'TYPE',
 'PERIODE',
 'QTE_PREV',
 'QTE_PREV_EQ_BLE']

In [68]:
produit_df.columns.to_list()

['CODE_PRODUIT',
 'RENDEMENT',
 'CODE_PRODUIT_STOCKAGE',
 'PRODUIT',
 'PERIODE',
 'TYPE',
 'MASSE_VOL']

In [31]:
# Ensure the relevant columns are in numeric format
collecte_prev_agg_df['QTE_PREV_EQ_BLE'] = pd.to_numeric(collecte_prev_agg_df['QTE_PREV_EQ_BLE'], errors='coerce')

# Left join Collecte prévisionnelle agg, cout Transfert, Produit on CODE_SITE and CODE_PRODUIT
merged_df = pd.merge(cout_transfert_df, produit_df, on=['CODE_PRODUIT'])
merged_df = pd.merge(merged_df, collecte_prev_agg_df, left_on=['CODE_PRODUIT_STOCKAGE', 'CODE_SITE_DESTINATION'], right_on=['CODE_PRODUIT_STOCKAGE', 'CODE_SITE'])

merged_df.columns
# Create "Quantité à stocker"
#merged_df['QUANTITE_A_STOCKER'] = merged_df['QTE_PREV_EQ_BLE'] + (merged_df['QUANTITE_TRANSFEREE'] / merged_df['MASSE_VOL'])

# Aggregation: CODE_PRODUIT_STOCKAGE, CODE_SITE, sum(QUANTITE_A_STOCKER)
#aggregated_df = merged_df.groupby(['CODE_PRODUIT_STOCKAGE', 'CODE_SITE'])['QUANTITE_A_STOCKER'].sum().reset_index()


Index(['CODE_PRODUIT', 'CODE_SITE_ORIGINE', 'CODE_SITE_DESTINATION',
       'QUANTITE_TRANSFEREE', 'COUT_UNITAIRE', 'COUT_TRANSFERT', 'RENDEMENT',
       'CODE_PRODUIT_STOCKAGE', 'PRODUIT', 'PERIODE_x', 'TYPE_x',
       'MASSE_VOL_x', 'CODE_SITE', 'MASSE_VOL_y', 'TYPE_y', 'PERIODE_y',
       'QTE_PREV', 'QTE_PREV_EQ_BLE'],
      dtype='object')

In [33]:
# Merge the DataFrames using suffixes to avoid generating '_x' and '_y'
merged_df = pd.merge(cout_transfert_df, produit_df, on=['CODE_PRODUIT'], suffixes=('', '_DROP'))
merged_df = pd.merge(merged_df, collecte_prev_agg_df, left_on=['CODE_PRODUIT_STOCKAGE', 'CODE_SITE_DESTINATION'], right_on=['CODE_PRODUIT_STOCKAGE', 'CODE_SITE'], suffixes=('', '_DROP'))

# Drop the columns with '_DROP' suffix
#cout_transfert_produit_collecte_df = merged_df[[col for col in merged_df.columns if not col.endswith('_DROP')]]
# Check the columns to ensure they are correctly merged
#cout_transfert_produit_collecte_df



,CODE_PRODUIT,CODE_SITE_ORIGINE,CODE_SITE_DESTINATION,QUANTITE_TRANSFEREE,COUT_UNITAIRE,COUT_TRANSFERT,RENDEMENT,CODE_PRODUIT_STOCKAGE,PRODUIT,PERIODE,TYPE,MASSE_VOL,CODE_SITE,QTE_PREV,QTE_PREV_EQ_BLE
0,CO0501,D700,D707,180,7.06,1271.0,5.0,CO0501,AVOINE BLANCHE,SUM,CONV,1.50,D707,271.97,181.0
1,CO0501,D020,D631,42,3.43,144.0,5.0,CO0501,AVOINE BLANCHE,SUM,CONV,1.50,D631,85.45,57.0
2,CO0501,D404,D631,90,3.06,275.0,5.0,CO0501,AVOINE BLANCHE,SUM,CONV,1.50,D631,85.45,57.0
3,CO0501,D701,D707,46,5.20,239.0,5.0,CO0501,AVOINE BLANCHE,SUM,CONV,1.50,D707,271.97,181.0
4,CO0501,D702,D707,9,5.20,47.0,5.0,CO0501,AVOINE BLANCHE,SUM,CONV,1.50,D707,271.97,181.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,CO0700,D635,D631,163,4.52,737.0,3.0,CO0700,COLZA,SUM,CONV,1.15,D631,94.23,82.0
461,CO0700,D143,D217,574,4.24,2434.0,3.0,CO0700,COLZA,SUM,CONV,1.15,D217,453.03,394.0
462,CO0700,D108,D201,135,3.06,413.0,3.0,CO0700,COLZA,SUM,CONV,1.15,D201,392.76,342.0
463,CO0700,D511,D508,260,6.70,1742.0,3.0,CO0700,COLZA,SUM,CONV,1.15,D508,1254.87,1091.0


In [36]:
# Create "Quantité à stocker"
merged_df['QUANTITE_A_STOCKER'] = (merged_df['QTE_PREV_EQ_BLE'] + (merged_df['QUANTITE_TRANSFEREE'] / merged_df['MASSE_VOL'])).round(0)

# Aggregation: CODE_PRODUIT_STOCKAGE, CODE_SITE, sum(QUANTITE_A_STOCKER)
quantite_a_stocker_df = merged_df.groupby(['CODE_PRODUIT_STOCKAGE', 'CODE_SITE'])['QUANTITE_A_STOCKER'].sum().reset_index()

In [37]:
quantite_a_stocker_df

,CODE_PRODUIT_STOCKAGE,CODE_SITE,QUANTITE_A_STOCKER
0,CO0100,D010,600.0
1,CO0100,D026,49245.0
2,CO0100,D059,1330.0
3,CO0100,D143,18567.0
4,CO0100,D216,8422.0
...,...,...,...
77,CO0700,D638,882.0
78,CO0700,D641,703.0
79,CO0700,D657,3039.0
80,CO0700,D707,2379.0


In [38]:
quantite_a_stocker_df.to_csv('/workspace/ocealia/data/out_quantite_a_stocker.csv', index=False, sep=';')

### 6.	Construction du plan / Reste à stocker

In [39]:
capacite_stockage_df.columns.to_list()

['CODE_PRODUIT_STOCKAGE', 'CODE_SITE', 'CAPACITE_DE_STOCKAGE', 'PERIODE']

In [40]:
quantite_a_stocker_df.columns.to_list()

['CODE_PRODUIT_STOCKAGE', 'CODE_SITE', 'QUANTITE_A_STOCKER']

In [41]:
reste_a_stocker_df['CAPACITE_DE_STOCKAGE'].fillna(0, inplace=True)


/tmp/ipykernel_11506/573401579.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  capacite_stockage_df['CAPACITE_DE_STOCKAGE'].fillna(0, inplace=True)


In [45]:
reste_a_stocker_df = pd.merge(quantite_a_stocker_df, capacite_stockage_df, on=['CODE_PRODUIT_STOCKAGE', 'CODE_SITE'], how='left')
reste_a_stocker_df['CAPACITE_DE_STOCKAGE'].fillna(0, inplace=True)
reste_a_stocker_df['RESTE_A_STOCKER'] = (reste_a_stocker_df['QUANTITE_A_STOCKER'] - reste_a_stocker_df['CAPACITE_DE_STOCKAGE']).round(0)
reste_a_stocker_df

/tmp/ipykernel_11506/3694687578.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  reste_a_stocker_df['CAPACITE_DE_STOCKAGE'].fillna(0, inplace=True)


,CODE_PRODUIT_STOCKAGE,CODE_SITE,QUANTITE_A_STOCKER,CAPACITE_DE_STOCKAGE,PERIODE,RESTE_A_STOCKER
0,CO0100,D010,600.0,1800.0,ETE,-1200.0
1,CO0100,D026,49245.0,14400.0,ETE,34845.0
2,CO0100,D059,1330.0,0.0,NaN,1330.0
3,CO0100,D143,18567.0,0.0,NaN,18567.0
4,CO0100,D216,8422.0,0.0,NaN,8422.0
...,...,...,...,...,...,...
78,CO0700,D638,882.0,0.0,NaN,882.0
79,CO0700,D641,703.0,0.0,NaN,703.0
80,CO0700,D657,3039.0,2600.0,ETE,439.0
81,CO0700,D707,2379.0,0.0,NaN,2379.0


In [49]:
reste_a_stocker_df = pd.merge(
    reste_a_stocker_df,
    produit_df[['CODE_PRODUIT_STOCKAGE', 'PERIODE']],  # Selecting only necessary columns for the merge
    on='CODE_PRODUIT_STOCKAGE',
    how='left'
)

reste_a_stocker_df.drop(columns=['PERIODE_x'], inplace=True)

# Renaming 'PERIODE_y' to 'PERIOD'
reste_a_stocker_df.rename(columns={'PERIODE_y': 'PERIODE'}, inplace=True)

reste_a_stocker_df

,CODE_PRODUIT_STOCKAGE,CODE_SITE,QUANTITE_A_STOCKER,CAPACITE_DE_STOCKAGE,RESTE_A_STOCKER,PERIODE
0,CO0100,D010,600.0,1800.0,-1200.0,SUM
1,CO0100,D010,600.0,1800.0,-1200.0,SUM
2,CO0100,D026,49245.0,14400.0,34845.0,SUM
3,CO0100,D026,49245.0,14400.0,34845.0,SUM
4,CO0100,D059,1330.0,0.0,1330.0,SUM
...,...,...,...,...,...,...
120,CO0700,D638,882.0,0.0,882.0,SUM
121,CO0700,D641,703.0,0.0,703.0,SUM
122,CO0700,D657,3039.0,2600.0,439.0,SUM
123,CO0700,D707,2379.0,0.0,2379.0,SUM


In [24]:
reste_a_stocker_df.to_csv('/workspace/ocealia/data/out_reste_a_stocker.csv', index=False, sep=';')

### **STAGING**

In [30]:
import pandas as pd
from io import StringIO
import os 

# ATTENTION : Remplacer toutes les virgules par des points dans les fichiers csv
folder_path = '/workspace/ocealia/data'

df_dict = {}

# Read all CSV files in the folder and assign them to variables based on filenames
for filename in os.listdir(folder_path):
    if filename.startswith('out'):
        var_name = os.path.splitext(filename)[0]
        df= pd.read_csv(os.path.join(folder_path, filename),sep=';')
        globals()[var_name+'_df'] = df
        df_dict[var_name] = df

for name, df in df_dict.items():
    print(name)
    print(*df.columns.tolist(), sep="\n")
    print("\n")



out_collecte_prev_eq_ble
CODE_PRODUIT_STOCKAGE
CODE_SITE
MASSE_VOL
TYPE
PERIODE
QTE_PREV
QTE_PREV_EQ_BLE


out_cout_transfert
CODE_PRODUIT
CODE_SITE_ORIGINE
CODE_SITE_DESTINATION
QUANTITE_TRANSFEREE
COUT
COUT_TRANSFERT


out_capacite_stockage
CODE_PRODUIT_STOCKAGE
CODE_SITE
CAPACITE_DE_STOCKAGE
PERIODE


out_quantite_a_stocker
CODE_PRODUIT_STOCKAGE
CODE_SITE
QUANTITE_A_STOCKER


out_reste_a_stocker
CODE_PRODUIT_STOCKAGE
CODE_SITE
QUANTITE_A_STOCKER
CAPACITE_DE_STOCKAGE
PERIODE
RESTE_A_STOCKER




<!DOCTYPE html>
<html>
<head>
  <style>
    body {
      font-family: Arial, sans-serif;
    }
    table {
      border-collapse: collapse;
      width: 100%;
      margin-bottom: 20px;
    }
    th {
      background-color: #4CAF50;
      color: white;
      padding: 8px;
    }
    td {
      border: 1px solid #ddd;
      padding: 8px;
      vertical-align: top;
    }
    td table {
      width: 100%;
      border: none;
    }
    td table td {
      border: none;
      padding: 4px;
    }
  </style>
</head>
<body>

<table>
  <tr>
    <td>
      <table>
        <tr><th>cellule</th></tr>
        <tr><td>CODE_SITE</td></tr>
        <tr><td>CODE_CELLULE</td></tr>
        <tr><td>TYPE_CELLULE</td></tr>
        <tr><td>VENTILATION</td></tr>
        <tr><td>CAPACITE</td></tr>
      </table>
    </td>
    <td>
      <table>
        <tr><th>cout_transport</th></tr>
        <tr><td>CODE_SITE_ORIGINE</td></tr>
        <tr><td>CODE_SITE_DESTINATION</td></tr>
        <tr><td>COUT</td></tr>
      </table>
    </td>
    <td>
      <table>
        <tr><th>plan_affectation</th></tr>
        <tr><td>CODE_DIVISION</td></tr>
        <tr><td>CODE_CELLULE</td></tr>
        <tr><td>CODE_PRODUIT_STOCKAGE_ETE</td></tr>
        <tr><td>CODE_PRODUIT_STOCKAGE_AUTOMNE</td></tr>
      </table>
    </td>
    <td>
      <table>
        <tr><th>plan_transfert</th></tr>
        <tr><td>CODE_PRODUIT</td></tr>
        <tr><td>CODE_SITE_ORIGINE</td></tr>
        <tr><td>CODE_SITE_DESTINATION</td></tr>
        <tr><td>QUANTITE_TRANSFEREE</td></tr>
      </table>
    </td>
  </tr>
  <tr>
    <td>
      <table>
        <tr><th>prevision_collecte</th></tr>
        <tr><td>CODE_PRODUIT</td></tr>
        <tr><td>CODE_ADHERENT</td></tr>
        <tr><td>CODE_SITE</td></tr>
        <tr><td>SURFACE_PREVISIONNELLE</td></tr>
      </table>
    </td>
    <td>
      <table>
        <tr><th>produit</th></tr>
        <tr><td>CODE_PRODUIT</td></tr>
        <tr><td>RENDEMENT</td></tr>
        <tr><td>CODE_PRODUIT_STOCKAGE</td></tr>
        <tr><td>PRODUIT</td></tr>
        <tr><td>PERIODE</td></tr>
        <tr><td>TYPE</td></tr>
        <tr><td>MASSE_VOL</td></tr>
      </table>
    </td>
    <td>
      <table>
        <tr><th>site</th></tr>
        <tr><td>CODE_SITE</td></tr>
        <tr><td>NOM_SITE</td></tr>
        <tr><td>SECTEUR</td></tr>
      </table>
    </td>
    <td>
      <table>
        <tr><th>out_collecte_prev_eq_ble</th></tr>
        <tr><td>CODE_PRODUIT_STOCKAGE</td></tr>
        <tr><td>CODE_SITE</td></tr>
        <tr><td>MASSE_VOL</td></tr>
        <tr><td>TYPE</td></tr>
        <tr><td>PERIODE</td></tr>
        <tr><td>QTE_PREV</td></tr>
        <tr><td>QTE_PREV_EQ_BLE</td></tr>
      </table>
    </td>
  </tr>
  <tr>
    <td>
      <table>
        <tr><th>out_cout_transfert</th></tr>
        <tr><td>CODE_PRODUIT</td></tr>
        <tr><td>CODE_SITE_ORIGINE</td></tr>
        <tr><td>CODE_SITE_DESTINATION</td></tr>
        <tr><td>QUANTITE_TRANSFEREE</td></tr>
        <tr><td>COUT_UNITAIRE</td></tr>
        <tr><td>COUT_TRANSFERT</td></tr>
      </table>
    </td>
    <td>
      <table>
        <tr><th>out_capacite_stockage</th></tr>
        <tr><td>CODE_PRODUIT_STOCKAGE</td></tr>
        <tr><td>CODE_SITE</td></tr>
        <tr><td>CAPACITE_DE_STOCKAGE</td></tr>
        <tr><td>PERIODE</td></tr>
      </table>
    </td>
    <td>
      <table>
        <tr><th>out_quantite_a_stocker</th></tr>
        <tr><td>CODE_PRODUIT_STOCKAGE</td></tr>
        <tr><td>CODE_SITE</td></tr>
        <tr><td>CAPACITE_DE_STOCKAGE</td></tr>
        <tr><td>PERIODE</td></tr>
      </table>
    </td>
    <td>
      <table>
        <tr><th>out_reste_a_stocker</th></tr>
        <tr><td>CODE_PRODUIT_STOCKAGE</td></tr>
        <tr><td>CODE_SITE</td></tr>
        <tr><td>QUANTITE_A_STOCKER</td></tr>
        <tr><td>CAPACITE_DE_STOCKAGE</td></tr>
        <tr><td>PERIODE</td></tr>
        <tr><td>RESTE_A_STOCKER</td></tr>
      </table>
    </td>
  </tr>
</table>

</body>
</html>


### SYNTHESE PAR PRODUIT

In [52]:
out_reste_a_stocker_df.columns

Index(['CODE_PRODUIT_STOCKAGE', 'CODE_SITE', 'QUANTITE_A_STOCKER',
       'CAPACITE_DE_STOCKAGE', 'PERIODE', 'RESTE_A_STOCKER'],
      dtype='object')

In [56]:
merged_df = produit_df.merge(out_reste_a_stocker_df, on='CODE_PRODUIT_STOCKAGE')
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.display.float_format = '{:,.2f}'.format


merged_df = out_reste_a_stocker_df.groupby(['CODE_PRODUIT_STOCKAGE', 'PERIODE'])[
    ['QUANTITE_A_STOCKER', 'CAPACITE_DE_STOCKAGE', 'RESTE_A_STOCKER']
].sum().reset_index()
 
merged_df

,CODE_PRODUIT_STOCKAGE,PERIODE,QUANTITE_A_STOCKER,CAPACITE_DE_STOCKAGE,RESTE_A_STOCKER
0,CO0100,ETE,"300,839.39","3,819,718.00","-3,518,878.61"
1,CO0133,ETE,"4,985.39","8,750.00","-3,764.61"
2,CO0152,AUT,"8,086.28","4,510.00","3,576.28"
3,CO0152,ETE,"8,339.70","41,390.00","-33,050.30"
4,CO0154,ETE,370.13,"1,760.00","-1,389.87"
5,CO0175,ETE,406.93,650.00,-243.07
6,CO0200,ETE,"19,612.01","211,080.00","-191,467.99"
7,CO0501,ETE,"5,454.24","25,630.00","-20,175.76"
8,CO0502,ETE,413.63,"20,500.00","-20,086.37"
9,CO0700,ETE,"68,318.03","694,780.00","-626,461.97"


In [57]:
# Assuming the data is already loaded in the following dataframes:
# produit_df, out_reste_a_stocker_df, out_cout_transfert_df

# Join produit_df with out_reste_a_stocker_df on CODE_PRODUIT_STOCKAGE
merged_df = produit_df.merge(out_reste_a_stocker_df, on='CODE_PRODUIT_STOCKAGE')

merged_df = out_reste_a_stocker_df.groupby(['CODE_PRODUIT_STOCKAGE', 'PERIODE'])[
    ['QUANTITE_A_STOCKER', 'CAPACITE_DE_STOCKAGE', 'RESTE_A_STOCKER']
].sum().reset_index()
# Add an additional join to include out_cout_transfert_df based on CODE_PRODUIT
final_df = merged_df.merge(out_cout_transfert_df, on='CODE_PRODUIT')

# Filter to ensure we get the correct PERIODE for CAPACITE STOCKAGE
#ete_df = out_reste_a_stocker_df[out_reste_a_stocker_df['PERIODE'] == 'SUM']
#aut_df = out_reste_a_stocker_df[out_reste_a_stocker_df['PERIODE'] == 'AUT']

# Aggregate CAPACITE_DE_STOCKAGE for ETE and AUT
#ete_capacite = ete_df[['CODE_PRODUIT_STOCKAGE', 'CAPACITE_DE_STOCKAGE']].rename(columns={'CAPACITE_DE_STOCKAGE': 'CAPACITE_STOCKAGE_ETE'})
#aut_capacite = aut_df[['CODE_PRODUIT_STOCKAGE', 'CAPACITE_DE_STOCKAGE']].rename(columns={'CAPACITE_DE_STOCKAGE': 'CAPACITE_STOCKAGE_AUT'})

# Merge capacities with the final dataframe
#final_df = final_df.merge(ete_capacite, on='CODE_PRODUIT_STOCKAGE', how='left').merge(aut_capacite, on='CODE_PRODUIT_STOCKAGE', how='left')

# Calculate total CAPACITE STOCKAGE
#final_df['CAPACITE_STOCKAGE_ETE_AUT'] = final_df['CAPACITE_STOCKAGE_ETE'].fillna(0) + final_df['CAPACITE_STOCKAGE_AUT'].fillna(0)

# Calculate COUT_TRANSFERT / QTE_TRANSFEREE
final_df['COUT_UNITAIRE_MOYEN'] = (final_df['COUT_TRANSFERT'] / final_df['QUANTITE_TRANSFEREE']).round(2)

# Select and rename columns to match the desired output
synthese_produit_df = final_df[['CODE_PRODUIT', 'PRODUIT', 'PERIODE_x', 'QUANTITE_A_STOCKER', 'CAPACITE_STOCKAGE_ETE_AUT', 'RESTE_A_STOCKER', 'QUANTITE_TRANSFEREE', 'COUT_TRANSFERT', 'COUT_UNITAIRE_MOYEN']]
synthese_produit_df = synthese_produit_df.rename(columns={
    'PERIODE_x': 'PERIODE',
})

# Display the final dataframe

synthese_produit_df


KeyError: 'CODE_PRODUIT'

In [56]:
synthese_produit_df.columns.tolist()

['CODE_PRODUIT',
 'PRODUIT',
 'PERIODE',
 'QUANTITE_A_STOCKER',
 'CAPACITE_STOCKAGE_ETE_AUT',
 'RESTE_A_STOCKER',
 'QUANTITE_TRANSFEREE',
 'COUT_TRANSFERT',
 'COUT_UNITAIRE_MOYEN']

In [61]:
synthese_produit_agreg_df = synthese_produit_df.groupby(['CODE_PRODUIT', 'PRODUIT']).agg({
    'QUANTITE_A_STOCKER': 'sum',
    'CAPACITE_STOCKAGE_ETE_AUT': 'sum',
    'RESTE_A_STOCKER': 'sum',
    'QUANTITE_TRANSFEREE': 'sum',
    'COUT_TRANSFERT': 'sum',
    'COUT_UNITAIRE_MOYEN': 'mean'
}).reset_index()

In [65]:
synthese_produit_agreg_df = synthese_produit_agreg_df.round(2)


In [71]:
pd.set_option('display.float_format', lambda x: '{:,.2f}'.format(x).replace(',', ' '))


In [72]:
synthese_produit_agreg_df

,CODE_PRODUIT,PRODUIT,QUANTITE_A_STOCKER,CAPACITE_STOCKAGE_ETE_AUT,RESTE_A_STOCKER,QUANTITE_TRANSFEREE,COUT_TRANSFERT,COUT_UNITAIRE_MOYEN
0,CO0100,BLE TENDRE,6 300 805 298.52,85 026 922 680.00,-55 950 169 846.53,3092838000,18 025 707 840.00,5.83
1,CO0133,BLE MELANGE VRM CRC,79 737.83,175 000.00,-56 469.21,22260,96 072.24,4.52
2,CO0152,BLE NFV 30-001,2 956 676.60,15 236 910.00,-5 305 323.40,723060,3 365 713.80,4.76
3,CO0154,BLE ARKEOS NF V30-001,8 527.40,36 960.00,-19 458.15,4494,28 268.88,7.36
4,CO0166,BLE DE FORCE IZALCO CS,8 926.06,0.00,0.00,3627,18 381.93,5.22
5,CO0173,BLE MEUNIER EXPORT,285 381.56,0.00,0.00,179144,761 759.32,4.76
6,CO0175,BLE DE FORCE CRC IZALCO CS,7 389.89,7 800.00,-972.29,1827,8 754.27,4.59
7,CO0200,BLE DUR,170 264 396.28,1 626 160 320.00,-901 431 296.92,75464928,332 200 566.72,4.58
8,CO0501,AVOINE BLANCHE,4 973 616.05,20 068 290.00,-10 531 746.62,872370,5 794 995.96,6.61
9,CO0502,AVOINE NOIRE,1 621 015.20,27 552 000.00,-10 123 528.87,684288,3 898 184.40,6.41


In [73]:
synthese_produit_agreg_df.to_csv('/workspace/ocealia/data/stage_synthese_produit.csv', index=False, sep=';')

In [49]:
synthese_produit_df.count()

CODE_PRODUIT                 5416512
PRODUIT                      5416512
PERIODE                      5416512
QUANTITE_A_STOCKER           5416512
CAPACITE_STOCKAGE_ETE_AUT    5416512
RESTE_A_STOCKER              3518245
QUANTITE_TRANSFEREE          5416512
COUT_TRANSFERT               5356080
COUT_UNITAIRE_MOYEN          5328080
dtype: int64

In [53]:
distinct_df = synthese_produit_df['CODE_PRODUIT'].unique()
distinct_df


array(['CO0501', 'CO0504', 'CO0502', 'CO0154', 'CO0175', 'CO0166',
       'CO0200', 'CO0133', 'CO0173', 'CO0152', 'CO0100', 'CO0700'],
      dtype=object)

In [1]:
distinct_df = produit_df['CODE_PRODUIT'].unique()
distinct_df

NameError: name 'produit_df' is not defined

In [5]:
# Site information
result_df = site_df[['CODE_SITE', 'NOM_SITE', 'SECTEUR']]

# Collecte
collecte_df = out_collecte_prev_eq_ble_df[['CODE_SITE', 'QTE_PREV_EQ_BLE']]
result_df = result_df.merge(collecte_df, on='CODE_SITE', how='left')

# Transfert Entrant
transfert_entrant_df = plan_transfert_df.groupby('CODE_SITE_DESTINATION').agg({'QUANTITE_TRANSFEREE': 'sum'}).reset_index()
transfert_entrant_df = transfert_entrant_df.rename(columns={'CODE_SITE_DESTINATION': 'CODE_SITE', 'QUANTITE_TRANSFEREE': 'TRANSFERT_ENTRANT'})
result_df = result_df.merge(transfert_entrant_df, on='CODE_SITE', how='left')

# Transfert Sortant
transfert_sortant_df = plan_transfert_df.groupby('CODE_SITE_ORIGINE').agg({'QUANTITE_TRANSFEREE': 'sum'}).reset_index()
transfert_sortant_df = transfert_sortant_df.rename(columns={'CODE_SITE_ORIGINE': 'CODE_SITE', 'QUANTITE_TRANSFEREE': 'TRANSFERT_SORTANT'})
result_df = result_df.merge(transfert_sortant_df, on='CODE_SITE', how='left')

# Capacité
capacite_df = out_capacite_stockage_df.groupby('CODE_SITE').agg({'CAPACITE_DE_STOCKAGE': 'sum'}).reset_index()
capacite_df = capacite_df.rename(columns={'CAPACITE_DE_STOCKAGE': 'CAPACITE'})
result_df = result_df.merge(capacite_df, on='CODE_SITE', how='left')

# Ecart
ecart_df = out_reste_a_stocker_df.groupby('CODE_SITE').agg({'RESTE_A_STOCKER': 'sum'}).reset_index()
ecart_df = ecart_df.rename(columns={'RESTE_A_STOCKER': 'ECART'})
result_df = result_df.merge(ecart_df, on='CODE_SITE', how='left')


In [6]:
result_df = result_df.dropna()
result_df

,CODE_SITE,NOM_SITE,SECTEUR,QTE_PREV_EQ_BLE,TRANSFERT_ENTRANT,TRANSFERT_SORTANT,CAPACITE,ECART
24,D107,ARCHIAC,JARNAC CHARENTE,478.4900,2164.0,66.0,223800.0,-11728.4348
25,D107,ARCHIAC,JARNAC CHARENTE,249.9500,2164.0,66.0,223800.0,-11728.4348
26,D107,ARCHIAC,JARNAC CHARENTE,8.9217,2164.0,66.0,223800.0,-11728.4348
27,D107,ARCHIAC,JARNAC CHARENTE,142.3565,2164.0,66.0,223800.0,-11728.4348
28,D107,ARCHIAC,JARNAC CHARENTE,1164.8565,2164.0,66.0,223800.0,-11728.4348
...,...,...,...,...,...,...,...,...
2278,D628,VIVONNE,ST MARTIN L ARS,249.0522,312.0,1142.0,223050.0,-112875.0659
2279,D628,VIVONNE,ST MARTIN L ARS,23.3700,312.0,1142.0,223050.0,-112875.0659
2280,D628,VIVONNE,ST MARTIN L ARS,101.4000,312.0,1142.0,223050.0,-112875.0659
2281,D628,VIVONNE,ST MARTIN L ARS,134.6957,312.0,1142.0,223050.0,-112875.0659


In [8]:
# Site information
result_df = site_df[['CODE_SITE', 'NOM_SITE', 'SECTEUR']]

# Collecte
collecte_df = out_collecte_prev_eq_ble_df[['CODE_SITE', 'CODE_PRODUIT_STOCKAGE', 'QTE_PREV_EQ_BLE']]
result_df = result_df.merge(collecte_df, on='CODE_SITE', how='left')

# Transfert Entrant
transfert_entrant_df = plan_transfert_df.groupby(['CODE_SITE_DESTINATION', 'CODE_PRODUIT']).agg({'QUANTITE_TRANSFEREE': 'sum'}).reset_index()
transfert_entrant_df = transfert_entrant_df.rename(columns={'CODE_SITE_DESTINATION': 'CODE_SITE', 'QUANTITE_TRANSFEREE': 'TRANSFERT_ENTRANT'})


In [10]:
result_df.columns

Index(['CODE_SITE', 'NOM_SITE', 'SECTEUR', 'CODE_PRODUIT_STOCKAGE',
       'QTE_PREV_EQ_BLE'],
      dtype='object')

In [11]:
transfert_entrant_df.columns

Index(['CODE_SITE', 'CODE_PRODUIT', 'TRANSFERT_ENTRANT'], dtype='object')

In [14]:

# Site information
result_df = site_df[['CODE_SITE', 'NOM_SITE', 'SECTEUR']]

# Collecte
collecte_df = out_collecte_prev_eq_ble_df[['CODE_SITE', 'CODE_PRODUIT_STOCKAGE', 'QTE_PREV_EQ_BLE']]
result_df = result_df.merge(collecte_df, on='CODE_SITE', how='left')

# Merge with produit to get CODE_PRODUIT for filters
result_df = result_df.merge(produit_df[['CODE_PRODUIT', 'CODE_PRODUIT_STOCKAGE']], on='CODE_PRODUIT_STOCKAGE', how='left')

# Transfert Entrant
transfert_entrant_df = plan_transfert_df.groupby(['CODE_SITE_DESTINATION', 'CODE_PRODUIT']).agg({'QUANTITE_TRANSFEREE': 'sum'}).reset_index()
transfert_entrant_df = transfert_entrant_df.rename(columns={'CODE_SITE_DESTINATION': 'CODE_SITE', 'QUANTITE_TRANSFEREE': 'TRANSFERT_ENTRANT'})
result_df = result_df.merge(transfert_entrant_df, on=['CODE_SITE', 'CODE_PRODUIT'], how='left')

# Transfert Sortant
transfert_sortant_df = plan_transfert_df.groupby(['CODE_SITE_ORIGINE', 'CODE_PRODUIT']).agg({'QUANTITE_TRANSFEREE': 'sum'}).reset_index()
transfert_sortant_df = transfert_sortant_df.rename(columns={'CODE_SITE_ORIGINE': 'CODE_SITE', 'QUANTITE_TRANSFEREE': 'TRANSFERT_SORTANT'})
result_df = result_df.merge(transfert_sortant_df, on=['CODE_SITE', 'CODE_PRODUIT'], how='left')

# Capacité
capacite_df = out_capacite_stockage_df.groupby(['CODE_SITE', 'CODE_PRODUIT_STOCKAGE']).agg({'CAPACITE_DE_STOCKAGE': 'sum'}).reset_index()
capacite_df = capacite_df.rename(columns={'CAPACITE_DE_STOCKAGE': 'CAPACITE'})
result_df = result_df.merge(capacite_df, on=['CODE_SITE', 'CODE_PRODUIT_STOCKAGE'], how='left')

# Ecart
ecart_df = out_reste_a_stocker_df.groupby(['CODE_SITE', 'CODE_PRODUIT_STOCKAGE']).agg({'RESTE_A_STOCKER': 'sum'}).reset_index()
ecart_df = ecart_df.rename(columns={'RESTE_A_STOCKER': 'ECART'})
result_df = result_df.merge(ecart_df, on=['CODE_SITE', 'CODE_PRODUIT_STOCKAGE'], how='left')

# Remove rows containing NaNs from result_df
stage_stockage_df = result_df.dropna()

In [15]:
stage_stockage_df 

,CODE_SITE,NOM_SITE,SECTEUR,CODE_PRODUIT_STOCKAGE,QTE_PREV_EQ_BLE,CODE_PRODUIT,TRANSFERT_ENTRANT,TRANSFERT_SORTANT,CAPACITE,ECART
42,D107,ARCHIAC,JARNAC CHARENTE,CO0700,36.7826,CO0700,789.0,40.0,11800.0,-11728.4348
160,D404,BEAUVAIS,TONNAY CHARENTE,CO0200,203.6000,CO0200,142.0,176.0,4400.0,-4020.4000
212,D605,BERNAC,VERTEUIL,CO0700,700.2261,CO0700,584.0,38.0,22200.0,-21466.7304
222,D405,BERNAY,TONNAY CHARENTE,CO0200,144.0000,CO0200,43.0,137.0,6410.0,-6129.0000
248,D702,BESSINES Silo new,BELLAC,CO0100,206.5000,CO0100,73.0,514.0,111250.0,-110529.5000
301,D440,BORDS,TONNAY CHARENTE,CO0100,705.1444,CO0100,931.0,831.0,21150.0,-19613.8556
502,D201,CHAMPAGNOLLES,ST GENIS DE SAINTONGE,CO0700,341.5304,CO0700,3095.0,1113.0,7980.0,-6670.6435
634,D641,CHATILLON,CHATILLON,CO0700,410.3217,CO0700,337.0,271.0,28550.0,-27904.0261
1803,D632,MONS,VERTEUIL,CO0100,3981.5206,CO0100,2735.0,1119.0,134960.0,-129859.4794
2058,D710,PARSAC,PARSAC,CO0502,6.6667,CO0502,6.0,20.0,1500.0,-1480.0000


In [ ]:
stage_stockage_df.withCo

In [28]:
stage_stockage_df.to_csv('/workspace/ocealia/data/stage_stockage_df.csv', index=False, sep=';')


In [8]:
import pandas as pd 
stage_stockage_df= pd.read_csv('/workspace/ocealia/data/stage_stockage_df.csv',sep=';')


In [3]:
stage_stockage_df.columns

Index(['CODE_SITE', 'NOM_SITE', 'SECTEUR', 'CODE_PRODUIT_STOCKAGE',
       'QTE_PREV_EQ_BLE', 'CODE_PRODUIT', 'TRANSFERT_ENTRANT',
       'TRANSFERT_SORTANT', 'CAPACITE', 'ECART', 'CODE_SITE_ORIGINE_x',
       'DEST_T1', 'DEST_T1_NOM_SITE', 'CODE_SITE_ORIGINE_y', 'COUT_T1',
       'QTE_T1'],
      dtype='object')

In [12]:
# Quantité à Transférer
stage_stockage_df['QUANTITE_A_TRANSFERER'] = stage_stockage_df['QTE_PREV_EQ_BLE']+ stage_stockage_df['TRANSFERT_ENTRANT'] - stage_stockage_df['CAPACITE']


In [16]:
# Dest T1
dest_t1_df = plan_transfert_df[['CODE_SITE_ORIGINE', 'CODE_SITE_DESTINATION', 'CODE_PRODUIT']]
dest_t1_df = dest_t1_df.merge(site_df[['CODE_SITE', 'NOM_SITE']], left_on='CODE_SITE_DESTINATION', right_on='CODE_SITE', how='left').rename(columns={'NOM_SITE': 'DEST_T1_NOM_SITE'})
stage_stockage_df = stage_stockage_df.merge(
    dest_t1_df[['CODE_SITE_ORIGINE', 'CODE_PRODUIT', 'CODE_SITE_DESTINATION', 'DEST_T1_NOM_SITE']],
    left_on=['CODE_SITE', 'CODE_PRODUIT'],
    right_on=['CODE_SITE_ORIGINE', 'CODE_PRODUIT'],
    how='left'
).rename(columns={'CODE_SITE_DESTINATION': 'DEST_T1'})

In [17]:
# Cout_T1 and Qte_T1
cout_qte_t1_df = out_cout_transfert_df[['CODE_SITE_ORIGINE', 'CODE_PRODUIT', 'COUT_TRANSFERT', 'QUANTITE_TRANSFEREE']]
stage_stockage_df = stage_stockage_df.merge(
    cout_qte_t1_df,
    left_on=['CODE_SITE', 'CODE_PRODUIT'],
    right_on=['CODE_SITE_ORIGINE', 'CODE_PRODUIT'],
    how='left'
).rename(columns={'COUT_TRANSFERT': 'COUT_T1', 'QUANTITE_TRANSFEREE': 'QTE_T1','CODE_SITE_ORIGINE_x':'CODE_SITE_ORIGINE'}).drop(columns=['CODE_SITE_ORIGINE_y'], inplace=True)
stage_stockage_df

In [13]:
stage_stockage_df

,CODE_SITE,NOM_SITE,SECTEUR,CODE_PRODUIT_STOCKAGE,QTE_PREV_EQ_BLE,CODE_PRODUIT,TRANSFERT_ENTRANT,TRANSFERT_SORTANT,CAPACITE,ECART,CODE_SITE_ORIGINE,DEST_T1,DEST_T1_NOM_SITE,COUT_T1,QTE_T1,QUANTITE_A_TRANSFERER
0,D107,ARCHIAC,JARNAC CHARENTE,CO0700,36.7826,CO0700,789.0,40.0,11800.0,-11728.4348,D107,D102,FONTAINES,148.00,40,-10974.2174
1,D404,BEAUVAIS,TONNAY CHARENTE,CO0200,203.6000,CO0200,142.0,176.0,4400.0,-4020.4000,D404,D632,MONS,538.56,176,-4054.4000
2,D605,BERNAC,VERTEUIL,CO0700,700.2261,CO0700,584.0,38.0,22200.0,-21466.7304,D605,D061,UCAP,130.34,38,-20915.7739
3,D405,BERNAY,TONNAY CHARENTE,CO0200,144.0000,CO0200,43.0,137.0,6410.0,-6129.0000,D405,D442,TONNAY CHARENTE SILO COOP,619.24,137,-6223.0000
4,D702,BESSINES Silo new,BELLAC,CO0100,206.5000,CO0100,73.0,514.0,111250.0,-110529.5000,D702,D703,DUN LE PALESTEL,2672.80,514,-110970.5000
5,D440,BORDS,TONNAY CHARENTE,CO0100,705.1444,CO0100,931.0,831.0,21150.0,-19613.8556,D440,D300,SICA LA PALLICE,5733.90,831,-19513.8556
6,D201,CHAMPAGNOLLES,ST GENIS DE SAINTONGE,CO0700,341.5304,CO0700,3095.0,1113.0,7980.0,-6670.6435,D201,D303,SILOS D'AQUITAINE,10284.12,1113,-4543.4696
7,D641,CHATILLON,CHATILLON,CO0700,410.3217,CO0700,337.0,271.0,28550.0,-27904.0261,D641,D314,OCERAIL,1596.19,271,-27802.6783
8,D632,MONS,VERTEUIL,CO0100,3981.5206,CO0100,2735.0,1119.0,134960.0,-129859.4794,D632,D631,CHIVES,3838.17,1119,-128243.4794
9,D710,PARSAC,PARSAC,CO0502,6.6667,CO0502,6.0,20.0,1500.0,-1480.0000,D710,D707,LIMOGES Silo,226.20,20,-1487.3333


In [10]:
stage_stockage_df = stage_stockage_df.rename(columns={'CODE_SITE_ORIGINE_x':'CODE_SITE_ORIGINE'}).drop(columns=['CODE_SITE_ORIGINE_y'])
stage_stockage_df

,CODE_SITE,NOM_SITE,SECTEUR,CODE_PRODUIT_STOCKAGE,QTE_PREV_EQ_BLE,CODE_PRODUIT,TRANSFERT_ENTRANT,TRANSFERT_SORTANT,CAPACITE,ECART,CODE_SITE_ORIGINE,DEST_T1,DEST_T1_NOM_SITE,COUT_T1,QTE_T1
0,D107,ARCHIAC,JARNAC CHARENTE,CO0700,36.7826,CO0700,789.0,40.0,11800.0,-11728.4348,D107,D102,FONTAINES,148.00,40
1,D404,BEAUVAIS,TONNAY CHARENTE,CO0200,203.6000,CO0200,142.0,176.0,4400.0,-4020.4000,D404,D632,MONS,538.56,176
2,D605,BERNAC,VERTEUIL,CO0700,700.2261,CO0700,584.0,38.0,22200.0,-21466.7304,D605,D061,UCAP,130.34,38
3,D405,BERNAY,TONNAY CHARENTE,CO0200,144.0000,CO0200,43.0,137.0,6410.0,-6129.0000,D405,D442,TONNAY CHARENTE SILO COOP,619.24,137
4,D702,BESSINES Silo new,BELLAC,CO0100,206.5000,CO0100,73.0,514.0,111250.0,-110529.5000,D702,D703,DUN LE PALESTEL,2672.80,514
5,D440,BORDS,TONNAY CHARENTE,CO0100,705.1444,CO0100,931.0,831.0,21150.0,-19613.8556,D440,D300,SICA LA PALLICE,5733.90,831
6,D201,CHAMPAGNOLLES,ST GENIS DE SAINTONGE,CO0700,341.5304,CO0700,3095.0,1113.0,7980.0,-6670.6435,D201,D303,SILOS D'AQUITAINE,10284.12,1113
7,D641,CHATILLON,CHATILLON,CO0700,410.3217,CO0700,337.0,271.0,28550.0,-27904.0261,D641,D314,OCERAIL,1596.19,271
8,D632,MONS,VERTEUIL,CO0100,3981.5206,CO0100,2735.0,1119.0,134960.0,-129859.4794,D632,D631,CHIVES,3838.17,1119
9,D710,PARSAC,PARSAC,CO0502,6.6667,CO0502,6.0,20.0,1500.0,-1480.0000,D710,D707,LIMOGES Silo,226.20,20


In [ ]:
stage_stockage_df['QUANTITE_A_TRANSFERER'] = stage_stockage_df['TRANSFERT_ENTRANT'] - stage_stockage_df['CAPACITE']


In [27]:
stage_stockage_df.columns.tolist()

['CODE_SITE',
 'NOM_SITE',
 'SECTEUR',
 'CODE_PRODUIT_STOCKAGE',
 'QTE_PREV_EQ_BLE',
 'CODE_PRODUIT',
 'TRANSFERT_ENTRANT',
 'TRANSFERT_SORTANT',
 'CAPACITE',
 'ECART',
 'CODE_SITE_ORIGINE',
 'QUANTITE_A_TRANSFERER',
 'DEST_T1',
 'DEST_T1_NOM_SITE',
 'COUT_T1',
 'QTE_T1']

In [26]:
stage_stockage_df = stage_stockage_df[['CODE_SITE',
 'NOM_SITE',
 'SECTEUR',
 'CODE_PRODUIT_STOCKAGE',
 'QTE_PREV_EQ_BLE',
 'CODE_PRODUIT',
 'TRANSFERT_ENTRANT',
 'TRANSFERT_SORTANT',
 'CAPACITE',
 'ECART',
 'CODE_SITE_ORIGINE',
 'QUANTITE_A_TRANSFERER',
 'DEST_T1',
 'DEST_T1_NOM_SITE',
 'COUT_T1',
 'QTE_T1']]